# Install packages 

In [ ]:
!pip install faiss

     |████████████████████████████████| 4.7MB 3.9MB/s 


In [ ]:
!sudo apt-get install libopenblas-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-dev is already the newest version (0.2.20+ds-4).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [ ]:
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 14 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (369 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty

In [ ]:
!pip install streamlit

     |████████████████████████████████| 7.4MB 3.8MB/s 
     |████████████████████████████████| 6.8MB 30.9MB/s 
     |████████████████████████████████| 112kB 47.7MB/s 
     |████████████████████████████████| 133kB 44.9MB/s 
     |████████████████████████████████| 163kB 43.2MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
     |████████████████████████████████| 4.5MB 39.1MB/s 
     |████████████████████████████████| 143kB 46.7MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 122kB 44.2MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=73d423866840d7198f00b2be5b310c80195f116ce8637c6c47a826c5cab60467
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=45fa56f031394ece3ff368f32717145a28c49b1a5

In [ ]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-5.0.0-cp36-none-any.whl size=18780 sha256=0f1d0dde4a8765484679b4f95d76ff0f903a5dfe3c58c1fe2320f259e6d9894c
  Stored in directory: /root/.cache/pip/wheels/95/df/23/af8dde08c3fcdc7b966adcacef48ab29aa3b0b1860df5d2b79
Successfully built pyngrok


# Read files from Gdrive (for users running this on google colab)

## copy embeddings file and index.py to current working directory

In [ ]:
!cp "/content/drive/My Drive/NTU/Year 4 Classes/Data Analytics/dnn_embeddings_sh_v2.csv" "dnn_embeddings_sh_v2.csv"

In [ ]:
!cp "/content/drive/My Drive/NTU/Year 4 Classes/Data Analytics/index.py" "index.py"

# Generate streamlit script

In [ ]:
%%writefile search_script.py
import streamlit as st
import pandas as pd
import numpy as np

from index import ExampleIndex

st.title('Player Search')

#Set up
embeddings_file_path = "dnn_embeddings_sh_v2.csv"
embeddings = pd.read_csv(embeddings_file_path)
embeddings_columns = [f'PC_{i}' for i in range(0,32)]


def get_dataset(df, embeddings_columns):
    vectors = np.ascontiguousarray(df[embeddings_columns])
    labels = np.ascontiguousarray(df['player_fifa_api_id'])

    return vectors.astype('float32'), np.expand_dims(labels, axis=1)

# Define Search Functions
class Service_Index():
    def __init__(self, df, embeddings_columns):
        vectors, labels = get_dataset(df, embeddings_columns)
        self.df = df
        self.vectors = vectors
        self.labels = labels

        self.index = ExampleIndex(vectors, labels)
        self.index.build()

        self.columns = ['player_name', 'overall_rating', 'potential', 'player_positions','birthday',
                        'player_fifa_api_id']

    def search_by_player_name(self, player_name):
        # player_name must be exact
        query_id = self.df[self.df.player_name == player_name].player_fifa_api_id.values[0]
        id_ = self.df[self.df.player_fifa_api_id == query_id].index.values[0]
        results, distances = self.index.query(np.expand_dims(self.vectors[id_], axis=0), k=11)
        response = []
        for ind, result_id in enumerate(results):
            if query_id != result_id:
                result_dict = self.df[self.df.player_fifa_api_id == result_id[0]][self.columns].iloc[0].to_dict()
                result_dict["similarity_score"] = 1/(1+distances[ind])
                response.append(result_dict)
        return response

    def search_by_vector(self, vector, query_id):
        results, distance = self.index.query(np.expand_dims(vector, axis=0), k=11)
        response = []
        for result_id in results:
          if result_id != query_id:
            response.append(self.df[self.df.player_fifa_api_id == result_id[0]][self.columns].iloc[0].to_dict())
        return response


# returns DF from list of dicts
def getDf(search_result, scores_only = False):
  def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

  result_df = pd.DataFrame()
  
  for result in search_result:
    result_df = result_df.append(result, ignore_index=True)
  if scores_only:
    result_df = result_df[['player_name', 'similarity_score','overall_rating', 'potential']]
    result_df = result_df.sort_values('similarity_score', ascending = False)
    columns_to_highlight = ['overall_rating', 'potential', 'similarity_score']
  else:
    if 'similarity_score' in result_df.columns:
      result_df = result_df[['player_name', 'overall_rating', 'potential', 'similarity_score', 'player_positions','birthday','player_fifa_api_id']]
    else:
      result_df = result_df[['player_name', 'overall_rating', 'potential', 'player_positions','birthday','player_fifa_api_id']]
    # result_df = result_df.sort_values('similarity_score', ascending = False)
    columns_to_highlight = ['overall_rating', 'potential']
  return result_df.style.apply(highlight_max, axis=0, subset=pd.IndexSlice[:,columns_to_highlight]) # 

def stSearchDb():
  st.sidebar.header("Use Case 1: Search all in DB")
  player_name = st.sidebar.text_input("Type player name", key = "1")

  if player_name:
    all_index = Service_Index(embeddings, embeddings_columns)
    search_result = all_index.search_by_player_name(player_name)
    st.dataframe(data=getDf(search_result))
    return

def stFilterSearch():
  embeddings.birthday = pd.to_datetime(embeddings.birthday)

  st.sidebar.header("Use Case 2: Filter-Search")
  # date_to_query = st.sidebar.text_input("Type earliest DOB in this format: YYYYMMDD (eg. 19950101)", key="2a")
  # year_input = st.sidebar.slider('Select earliest DOB', 1975, 2010, 1995, format = 'Birth Year %d')
  # date_to_query = str(year_input) + "0101"
  max_age = st.sidebar.slider('Select max age of related players', 10, 45, 21, format = '%d y.o.')
  year = 2016 - max_age
  date_to_query = str(year) + "0101"
  player_name = st.sidebar.text_input("Type player name", key="2b")

  if date_to_query and player_name:
    new_df = embeddings.query('birthday > ' + date_to_query)

    query_id = embeddings[embeddings.player_name == player_name].player_fifa_api_id.values[0]
    query_vector = np.array(embeddings[embeddings.player_fifa_api_id == query_id][embeddings_columns])
    query_vector = query_vector.reshape((32)).astype('float32')

    result_df = pd.DataFrame()
    index = Service_Index(new_df, embeddings_columns)
    response = index.search_by_vector(query_vector, query_id)
    st.dataframe(data=getDf(response))
    return

def stBestSearch():
  st.sidebar.header("Use Case 3: Search and compare similarity metrics")
  player_name = st.sidebar.text_input("Type player name", key="3")
  if player_name:
    all_index = Service_Index(embeddings, embeddings_columns)
    response = all_index.search_by_player_name(player_name)
    sorted_df = getDf(response, True)
    st.dataframe(data=sorted_df)
    return

def main():
  st.sidebar.title("Choose Options")
  st.subheader("Once an option is selected, a table of closely-related players will be generated. Important values are highlighted in yellow.")
  
  # st.header("USE CASE 1: Search all in DB")
  stSearchDb()

  # st.header("Use Case 2: Filter-Search")
  stFilterSearch()

  # st.header("Use Case 3: Best-Search")
  stBestSearch()



if __name__ == "__main__":
  main()



Writing search_script.py


# Run Streamlit script

In [ ]:
!streamlit run search_script.py &>/dev/null&

In [ ]:
# Required if you are running this notebook on remote server

from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect('8501')
public_url

<NgrokTunnel: "http://641ef5b0d68f.ngrok.io" -> "http://localhost:8501">

# (Optional) Generate requirements.txt for reproducibility

In [ ]:
!pip install pipreqs

     |████████████████████████████████| 133kB 6.7MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.2
    Uninstalling urllib3-1.26.2:
      Successfully uninstalled urllib3-1.26.2


In [ ]:
!touch requirements.txt

In [ ]:
!pipreqs .

ERROR: Failed on file: ./drive/My Drive/Intern/Ninjavan/Project Code/hr-integrations/vendor/github.com/ugorji/go/codec/test.py
Traceback (most recent call last):
  File "/usr/local/bin/pipreqs", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/pipreqs/pipreqs.py", line 470, in main
    init(args)
  File "/usr/local/lib/python3.6/dist-packages/pipreqs/pipreqs.py", line 409, in init
    follow_links=follow_links)
  File "/usr/local/lib/python3.6/dist-packages/pipreqs/pipreqs.py", line 138, in get_all_imports
    raise exc
  File "/usr/local/lib/python3.6/dist-packages/pipreqs/pipreqs.py", line 124, in get_all_imports
    tree = ast.parse(contents)
  File "/usr/lib/python3.6/ast.py", line 35, in parse
    return compile(source, filename, mode, PyCF_ONLY_AST)
  File "<unknown>", line 101
    print client.call("Echo123", "A1", "B2", "C3")
               ^
SyntaxError: invalid syntax


In [ ]:
from google.colab import files
files.download('requirements.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>